# Loading necessary libraries

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from sklearn.metrics import roc_auc_score
import re
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import  cross_val_score
import time
from sklearn.datasets import  load_iris
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
from sklearn.model_selection import KFold

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Loading dataset

In [3]:
data = pd.read_excel("NepaliEarthquakeTweets_plus_blockade.xlsx",sheetname=0)

In [4]:
pos_indexes = np.where(data['bhawna']=='p')
neg_indexes = np.where(data['bhawna']=='n')

In [5]:
len(neg_indexes[0])

605

In [7]:
pos_indexes = np.where(data['bhawna']=='p')
neg_indexes = np.where(data['bhawna']=='n')
size = 500
df_pos = pd.DataFrame.from_items([('text',data['tweet_text'][pos_indexes[0][0:size]]),('target',0)])
df_neg = pd.DataFrame.from_items([('text',data['tweet_text'][neg_indexes[0][0:size]]),('target',1)])

df_pos['tweet_id'] = data['tweet_id'][pos_indexes[0][0:size]]
df_pos['reply'] = data['reply'][pos_indexes[0][0:size]]
df_pos['retweet'] = data['retweet'][pos_indexes[0][0:size]]
df_pos['likes'] = data['likes'][pos_indexes[0][0:size]]
#df_pos['emoji'] = data['emoji'][pos_indexes[0][0:size]]
df_pos['bhawna']=data['bhawna'][pos_indexes[0][0:size]]

df_neg['tweet_id'] = data['tweet_id'][neg_indexes[0][0:size]]
df_neg['reply'] = data['reply'][neg_indexes[0][0:size]]
df_neg['retweet'] = data['retweet'][neg_indexes[0][0:size]]
df_neg['likes'] = data['likes'][neg_indexes[0][0:size]]
#df_neg['emoji'] = data['emoji'][neg_indexes[0][0:size]]
df_neg['bhawna']=data['bhawna'][neg_indexes[0][0:size]]


df_pos.index = range(len(df_pos.index))
df_neg.index = range(len(df_neg.index))

In [8]:
len(df_neg)

500

In [9]:
validation_df_F= df_pos[500:600]
validation_df_F = validation_df_F.append(df_neg[500:600])
validation_df_F.index = range(len(validation_df_F.index))

# Writing necessary Function

In [10]:
import re
def normalize(X):
    #print(X)
    if(max(X)==0 and min(X)==0):
        return X
    return [((x-min(X))/(max(X)-min(X))) for x in X]
#print(normalize(X))
def count_urls(text):
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    return len(urls)
def count_hashtags(text):
    hashtags = re.findall(r"#(\w+)", text)
    return len(hashtags)
def count_user_mentions(text):
    user_mentions = re.findall(r"@(\w+)", text)
    return len(user_mentions)
def count_unique_characters(text):
    return len(list(set(text)))
from string import punctuation
def count_special_characters(text):
    sc = [p for p in text if p in punctuation]
    return len(sc)
def get_words(text):
    words = text.replace(","," ").replace(". "," ").strip().split()
    return words
def count_words(text):
    words = get_words(text)
    return len(words)
slangwords = pd.read_excel('SlangWordsForNepali.xlsx')
for index in range(0,len(data['slangwords'])):
    w = data['slangwords'][index]
    w = str(w).rstrip().strip()
    if(w!='nan' and len(w)>0):
        words = w.split(",")
        for w1 in words:
            w1 = w1.rstrip().strip()
            if(w1 not in slangwords['Nepali'] and len(w1)>0):
                slangwords=slangwords.append({'Roman English':'','Nepali':w1,'English':'','Meaning':''},ignore_index=True)
def count_slangwords(text):
    sw = [w for w in get_words(text) if (len(np.where(slangwords['Nepali']==w)[0].tolist())>0)]
    return len(sw)
stopwords = pd.read_csv('nepali_stopwords.txt')
def count_stopwords(text):
    sw = [w for w in get_words(text) if (len(np.where(stopwords['stopwords']==w)[0].tolist())>0)]
    return len(sw)

In [11]:
slangwords = pd.read_excel('SlangWordsForNepali.xlsx')
for index in range(0,len(data['slangwords'])):
    w = data['slangwords'][index]
    w = str(w).rstrip().strip()
    if(w!='nan' and len(w)>0):
        words = w.split(",")
        for w1 in words:
            w1 = w1.rstrip().strip()
            if(w1 not in slangwords['Nepali'] and len(w1)>0):
                slangwords=slangwords.append({'Roman English':'','Nepali':w1,'English':'','Meaning':''},ignore_index=True)
len(slangwords)


102

# Loading Urdu to English Dictionary Database

In [14]:
import MySQLdb
db = MySQLdb.connect("localhost","root","","nepal_earthquak_2015_tweets",use_unicode=True, charset="utf8")
cursor = db.cursor()

In [15]:
dictionary = pd.read_excel("dictionary.xlsx",sheetname=0)

In [16]:
w='hello'
adjective = []
#if(len(exist)==1 and 'adjective' in dictionary['POS'][exist[0]]):
#    print("Word: "+w+" is adjective")
exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
adjective.append([w for index in exist if('adjective' in dictionary['POS'][index])])
#dictionary['POS'][exist]
adjective

[[]]

In [17]:
dictionary

,Roman Nepali,Nepali,Nepali Converted,POS,Meaning
0,aba,ca,अब,Adverb;,"now, from now on"
1,abaddha,cfa$,आबद्ध,Noun;/adjective;,"bound, tied up confined"
2,abela,ca]nf,अबेला,Noun;/adjective;,"lateness, late"
3,aber,ca]/,अबेर,adjective;,Late
4,aber bhayo,ca]/ eof],अबेर भयो,compound,it is late
5,abhas,cfef;,आभास,Noun;,"glimpse, shadow, reflection, vision, faint mem..."
6,abhav,cefj,अभाव,Noun;,"non-existence, absence, lack, scarcity"
7,abhaya,ceo,अभय,Noun;,lack of fear
8,abhibhavak,cleefjs,अभिभावक,Noun;,guardian
9,abhilekh,clen]v,अभिलेख,Noun;,inscription


# Writing function for formality feature

Formality = (#nouns + #adjectives + #prepositions + #articles - #pronouns- #verbs - #adverbs - #interjections + 100)/2

In [18]:
text = "एउटाले केन्द्र गोर्खा भन्छ/अर्कोले केन्द्र लमजुङ भन्छ! हैन योSLCको परिक्षा केन्द्र जस्तो हो कि क्या हो?एकैपटक जताकतै हुने?#NepalEarthquake"
#text = "आज बिहान ३:२३ मा गएको भुकम्प: ५.० रेक्टरस्केल केन्द्र गोरखा। #NepalEarthquake"
#print("Tweets: "+text)
def count_nouns(text):
    words = get_words(text)
    nouns = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%noun%' or pos like 'noun%' or pos like '%noun')"
        query1 = "SELECT * FROM `table 8` where word='"+w+ "' and pos = 'सङ्. ना.' or pos = 'असङ्. ना.' or pos ='ना.' or pos = 'संवा.'"
        #print("Query: "+query1)
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            nouns.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            #print(exist)
            #print("Word: "+w)
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'Noun' in dictionary['POS'][exist[0]]):
                nouns.append(w)'''
          #temp = [w for index in exist if('noun' in dictionary['POS'][index])]
                #nouns.append(tw)
    #print("Nouns:")
    #print(nouns)
    return(len(nouns))

def count_adjectives(text):
    words = get_words(text)
    adjectives = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        ##query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%adjective%' or pos like 'adjective%' or pos like '%adjective')"
        query1 = "SELECT * FROM `table 8` where word='"+w+ "' and pos like '%वि. ' or pos like '%वि.'" 
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            adjectives.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'adjective' in dictionary['POS'][exist[0]]):
                adjectives.append(w)'''
            #temp = [w for index in exist if('adjective' in dictionary['POS'][index])]
            #for tw in temp:
                #adjectives.append(tw)
    #print("Adjectives")
    #print(adjectives)
    return(len(adjectives))

def count_prepositions(text):
    words = get_words(text)
    prepositions = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition')"
        query1="SELECT * FROM `table 8` where word = '"+w+"' and pos =' नायो.' "
        #print("Query: "+query1)
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            prepositions.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and'preposition' in dictionary['POS'][exist[0]]):
                prepositions.append(w)
            #temp = [w for index in exist if('preposition' in dictionary['POS'][index])]
            #for tw in temp:
                #prepositions.append(tw)'''
    #print("Prepositions")
    #print(prepositions)
    return(len(prepositions))

def count_pronouns(text):
    words = get_words(text)
    pronouns = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun')"
        query1="SELECT * FROM `table 8` where word = '"+w+"' and pos like '% स.'"
        #print("Query: "+query1)
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            pronouns.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'pronoun' in dictionary['POS'][exist[0]]):
                pronouns.append()'''
            #temp = [w for index in exist if('pronoun' in dictionary['POS'][index])]
            #for tw in temp:
                #pronouns.append(tw)
    #print("Pronouns")
    #print(pronouns)
    return(len(pronouns))

def count_verbs(text):
    words = get_words(text)
    verbs = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%verb%' or pos like 'verb%' or pos like '%verb')"   
        query1 = "SELECT * FROM `table 8`  where word = '"+w+"' and  pos like '%क्रि.'"
        #print("Query: "+query1)
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            verbs.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'VERB ' in dictionary['POS'][exist[0]]):
                verbs.append(w)'''
    #print("Verbs")
    #print(verbs)
    return(len(verbs))

def count_adverbs(text):
    words = get_words(text)
    adverbs = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and not (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb')"
        query1 = "SELECT * FROM `table 8`  where word = '"+w+"' and  pos like '%क्रियो.' or pos like '%क्रियो. '"
        #print("Query: "+query1)
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            adverbs.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'adverb' in dictionary['POS'][exist[0]]):
                adverbs.append(w)'''
    #print("Adverbs")
    #print(adverbs)
    return(len(adverbs))

def count_conjunctions(text):
    words = get_words(text)
    conjunction = []
    for w in words:
        #query1 = "SELECT meaning,pos FROM `urdu_english_dictionary` where word ='"+w+"' and not (pos like '%noun%' or pos like 'noun%' or pos like '%noun') and not (pos like '%adjective%' or pos like '%adjective' or pos like 'adjective%') and not (pos like '%preposition%' or pos like 'preposition%' or pos like '%preposition') and not (pos like '%article%' or pos like 'article%' or pos like '%article') and not (pos like '%pronoun%' or pos like 'pronoun%' or pos like '%pronoun') and not (pos like '%verb%' or pos like 'verb%' or pos like '%verb') and not (pos like '%adverb%' or pos like 'adverb%' or pos like '%adverb') and (pos like '%interjection%' or pos like 'interjection%' or pos like '%interjection')"
        #query1 = "SELECT meaning,pos FROM `nepali_english_dictionary` where word ='"+w+"' and (pos like '%conjunction%' or pos like 'conjunction%' or pos like '%conjunction')"
        query1 = "SELECT * FROM `table 8`  where word = '"+w+"' and pos like '%सं.'"
        cursor.execute(query1)
        exist = cursor.fetchall()
        if(len(exist)>0):
            conjunction.append(w)
        '''if(len(exist)==0):
            exist = np.where(dictionary['Nepali Converted']==w)[0].tolist()
            if(len(exist)>0 and str(dictionary['POS'][exist[0]])!='nan' and 'conjunction' in dictionary['POS'][exist[0]]):
                conjunction.append(w)'''
    #print("conjunction")
    #print(conjunction)
    return(len(conjunction))
#count_nouns(text)            
#count_adjectives(text)  
#count_prepositions(text)
#count_pronouns(text)
#count_verbs(text)
#count_adverbs(text)
#count_conjunctions(text)

In [19]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
pat3 = r'([@#][A-Za-z0-9]+)'
pat4 = r'.[A-Za-z0-9./]+'
pat5 = r'[\,۔،۔”–’‘‘_!…।-]|(")|(:)|(%)|(ः)|(\u200d)|(\xa0…)|(\u200c\u200c)'
combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5))

df_pos['nouns']=[count_nouns(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]
df_pos['adjectives']=[count_adjectives(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]
df_pos['prepositions']=[count_prepositions(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]
df_pos['pronouns']=[count_pronouns(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]
df_pos['verbs']=[count_verbs(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]  
df_pos['adverb']=[count_adverbs(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]
df_pos['conjunction']=[count_conjunctions(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_pos['text']]

df_neg['nouns']=[count_nouns(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]
df_neg['adjectives']=[count_adjectives(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]
df_neg['prepositions']=[count_prepositions(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]
df_neg['pronouns']=[count_pronouns(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]
df_neg['verbs']=[count_verbs(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]  
df_neg['adverb']=[count_adverbs(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]
df_neg['conjunction']=[count_conjunctions(re.sub(combined_pat, ' ', text).strip().rstrip()) for text in df_neg['text']]

In [20]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [83]:
def feature_extraction(X_train,X_test):
    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    pat3 = r'([@#][A-Za-z0-9]+)'
    pat4 = r'.[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4))
    temp_train = [re.sub(combined_pat, '', x) for x in X_train['text']]
    temp_test = [re.sub(combined_pat, '', x) for x in X_test['text']]
    #vect = CountVectorizer(min_df=5, ngram_range=(1,1)).fit(temp_train)
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,4)).fit(temp_train)
    #vect = CountVectorizer(min_df=5, ngram_range=(1,3)).fit(temp_test)
    X_train_chars = X_train['text'].str.len()
    X_test_chars = X_test['text'].str.len()

    X_train_digit = X_train['text'].apply(lambda x:len(re.findall(r'(\d)',x)))
    X_test_digit = X_test['text'].apply(lambda x:len(re.findall(r'(\d)',x)))

    X_train_URL = normalize([count_urls(text) for text in X_train['text']])
    X_test_URL = normalize([count_urls(text) for text in X_test['text']])

    #print(X_train_URL)
    X_train_HASHTAGS = normalize([count_hashtags(text) for text in X_train['text']])
    X_test_HASHTAGS = normalize([count_hashtags(text) for text in X_test['text']])

    X_train_USERMENTION = normalize([count_user_mentions(text) for text in X_train['text']])
    X_test_USERMENTION = normalize([count_user_mentions(text) for text in X_test['text']])

    X_train_UC = normalize([count_unique_characters(text) for text in X_train['text']])
    X_test_UC = normalize([count_unique_characters(text) for text in X_test['text']])

    X_train_SC = normalize([count_special_characters(text) for text in X_train['text']])
    X_test_SC = normalize([count_special_characters(text) for text in X_test['text']])

    X_train_WORDS = normalize([count_words(text) for text in X_train['text']])
    X_test_WORDS = normalize([count_words(text) for text in X_test['text']])
    
    X_train_reply = normalize(X_train['reply'])
    X_test_reply = normalize(X_test['reply'])
    
    X_train_retweet = normalize(X_train['retweet'])
    X_test_retweet = normalize(X_test['retweet'])
    
    X_train_likes = normalize(X_train['likes'])
    X_test_likes = normalize(X_test['likes'])
    
    X_train_SW = normalize([count_slangwords(text) for text in X_train['text']])
    X_test_SW = normalize([count_slangwords(text) for text in X_test['text']])
    
    X_train_STOPWORDS = normalize([count_stopwords(text) for text in X_train['text']])
    X_test_STOPWORDS = normalize([count_stopwords(text) for text in X_test['text']])
    
    X_train_NOUNS = normalize(X_train['nouns'])
    X_test_NOUNS = normalize(X_test['nouns'])

    X_train_ADJECTIVES = normalize(X_train['adjectives'])
    X_test_ADJECTIVES = normalize(X_test['adjectives'])
    
    X_train_PREPOSITIONS = normalize(X_train['prepositions'])
    X_test_PREPOSITIONS = normalize(X_test['prepositions'])

    X_train_PRONOUNS = normalize(X_train['pronouns'])
    X_test_PRONOUNS = normalize(X_test['pronouns'])

    X_train_VERBS = normalize(X_train['verbs'])
    X_test_VERBS = normalize(X_test['verbs'])

    X_train_ADVERBS = normalize(X_train['adverb'])
    X_test_ADVERBS = normalize(X_test['adverb'])

    X_train_CONJUNCTIONS = normalize(X_train['conjunction'])
    X_test_CONJUNCTIONS = normalize(X_test['conjunction'])

    #print(X_train_WORDS)

    X_train_vectorized = vect.transform(X_train['text'])
    X_test_vectorized = vect.transform(X_test['text'])


    X_train_vectorized = add_feature(X_train_vectorized, [normalize(X_train_chars)])
    X_train_vectorized = add_feature(X_train_vectorized, [normalize(X_train_digit)])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_URL])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_HASHTAGS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_USERMENTION])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_UC])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_SC])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_WORDS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_reply])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_retweet])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_likes])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_SW])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_STOPWORDS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_NOUNS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_ADJECTIVES])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_PREPOSITIONS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_PRONOUNS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_VERBS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_ADVERBS])
    X_train_vectorized = add_feature(X_train_vectorized, [X_train_CONJUNCTIONS])

    X_test_vectorized = add_feature(X_test_vectorized, [normalize(X_test_chars)])
    X_test_vectorized = add_feature(X_test_vectorized, [normalize(X_test_digit)])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_URL])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_HASHTAGS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_USERMENTION])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_UC])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_SC])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_WORDS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_reply])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_retweet])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_likes])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_SW])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_STOPWORDS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_NOUNS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_ADJECTIVES])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_PREPOSITIONS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_PRONOUNS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_VERBS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_ADVERBS])
    X_test_vectorized = add_feature(X_test_vectorized, [X_test_CONJUNCTIONS])
    return (X_train_vectorized,X_test_vectorized)

In [84]:
def get_train_test(df_pos,df_neg,train_index,test_index):
    X_train = df_pos.iloc[train_index.tolist()]
    X_train = X_train.append(df_neg.iloc[train_index.tolist()])
    X_train.index = range(len(X_train.index))
    
    y_train = df_pos.target[train_index.tolist()]
    y_train = y_train.append(df_neg.target[train_index.tolist()])
    y_train.index = range(len(y_train.index))
    
    X_test = df_pos.iloc[test_index.tolist()]
    X_test = X_test.append(df_neg.iloc[test_index.tolist()])
    X_test.index = range(len(X_test.index))
    
    y_test = df_pos.target[test_index.tolist()]
    y_test = y_test.append(df_neg.target[test_index.tolist()])
    y_test.index = range(len(y_test.index))
    
    return(X_train,y_train,X_test,y_test)

In [85]:
kf = KFold(n_splits=10)
print(kf) 

KFold(n_splits=10, random_state=None, shuffle=False)


# Module For Classification

In [86]:
accuracy = []
precision= []
recall = []
roc = [] 
#df_pos = df_pos[0:700]
#df_neg = df_neg[0:700]
predictedPositive = []
predictedNegative = []
#counter = 0
for train_index, test_index in kf.split(df_pos):
    #counter = counter + 1
    #print("PRocessing fold "+str(counter))
    X_train,y_train,X_test,y_test = get_train_test(df_pos,df_neg,train_index,test_index)
    X_train_vectorized,X_test_vectorized = feature_extraction(X_train,X_test)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    roc.append(roc_auc_score(y_test, predictions))
    accuracy.append(accuracy_score(y_test, predictions, normalize=True))
    precision.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions))
    result = pd.DataFrame.from_items([("Actual",y_test),("Prediction",predictions)])
    pp = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==0))]
    nn = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==1))]
    predictedPositive.append(len(pp))
    predictedNegative.append(len(nn))    
a = sum(accuracy)/10
p = sum(precision)/10
r = sum(recall)/10
ro = sum(roc)/10
print("Accuracy: ",a,"\tPrecision: ",p,"\tRecall: ",r,"\tROC: ",ro)
TP = sum(predictedPositive)
FP = (len(df_pos)-sum(predictedPositive))
TN = sum(predictedNegative)
FN = (len(df_neg)-sum(predictedNegative))
print("Actual Positive: ",len(df_pos),"\tPredictedPositive(TP): ",TP,"\tFP: ",FP)
print("Actual Negative: ",len(df_neg),"\tPredictedNegative(TN): ",TN,"\tFN: ",FN)
F_measure = (2 * p * r)/(p+ r)
print("F-measure: "+str(F_measure))

Accuracy:  0.671 	Precision:  0.683470098702 	Recall:  0.66 	ROC:  0.671
Actual Positive:  500 	PredictedPositive(TP):  341 	FP:  159
Actual Negative:  500 	PredictedNegative(TN):  330 	FN:  170
F-measure: 0.671530040868


In [87]:
accuracy = []
precision= []
recall = []
roc = [] 
#df_pos = df_pos[0:700]
#df_neg = df_neg[0:700]
predictedPositive = []
predictedNegative = []
for train_index, test_index in kf.split(df_pos):
    X_train,y_train,X_test,y_test = get_train_test(df_pos,df_neg,train_index,test_index)
    X_train_vectorized,X_test_vectorized = feature_extraction(X_train,X_test)
    model = DecisionTreeClassifier()
    model.fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    roc.append(roc_auc_score(y_test, predictions))
    accuracy.append(accuracy_score(y_test, predictions, normalize=True))
    precision.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions))
    result = pd.DataFrame.from_items([("Actual",y_test),("Prediction",predictions)])
    pp = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==0))]
    nn = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==1))]
    predictedPositive.append(len(pp))
    predictedNegative.append(len(nn))    
a = sum(accuracy)/10
p = sum(precision)/10
r = sum(recall)/10
ro = sum(roc)/10
print("Accuracy: ",a,"\tPrecision: ",p,"\tRecall: ",r,"\tROC: ",ro)
TP = sum(predictedPositive)
FP = (len(df_pos)-sum(predictedPositive))
TN = sum(predictedNegative)
FN = (len(df_neg)-sum(predictedNegative))
print("Actual Positive: ",len(df_pos),"\tPredictedPositive(TP): ",TP,"\tFP: ",FP)
print("Actual Negative: ",len(df_neg),"\tPredictedNegative(TN): ",TN,"\tFN: ",FN)
F_measure = (2 * p * r)/(p+ r)
print("F-measure: "+str(F_measure))

Accuracy:  0.583 	Precision:  0.588661960324 	Recall:  0.562 	ROC:  0.583
Actual Positive:  500 	PredictedPositive(TP):  302 	FP:  198
Actual Negative:  500 	PredictedNegative(TN):  281 	FN:  219
F-measure: 0.575022088345


In [89]:
accuracy = []
precision= []
recall = []
roc = [] 
#df_pos = df_pos[0:700]
#df_neg = df_neg[0:700]
predictedPositive = []
predictedNegative = []
for train_index, test_index in kf.split(df_pos):
    X_train,y_train,X_test,y_test = get_train_test(df_pos,df_neg,train_index,test_index)
    X_train_vectorized,X_test_vectorized = feature_extraction(X_train,X_test)
    #model = SVC(C=10000)
    #model = SVC (C=10000, kernel='rbf', degree=3, gamma='auto')
    #model = SVC (C=1000, kernel='poly', degree=3, gamma='auto')
    #model = SVC (C=1000, kernel='linear', degree=3, gamma='auto')
    #model = SVC (C=1000, kernel='sigmoid', degree=3, gamma='auto')
    model = SVC (C=1.0,  kernel='rbf',gamma=0.10000000000000001)
    #model = SVC (C=100)
    #model = SVC (C=100, kernel='rbf', degree=3, gamma='auto')
    model.fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    roc.append(roc_auc_score(y_test, predictions))
    accuracy.append(accuracy_score(y_test, predictions, normalize=True))
    precision.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions))
    result = pd.DataFrame.from_items([("Actual",y_test),("Prediction",predictions)])
    pp = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==0))]
    nn = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==1))]
    predictedPositive.append(len(pp))
    predictedNegative.append(len(nn))    
a = sum(accuracy)/10
p = sum(precision)/10
r = sum(recall)/10
ro = sum(roc)/10
print("Accuracy: ",a,"\tPrecision: ",p,"\tRecall: ",r,"\tROC: ",ro)
TP = sum(predictedPositive)
FP = (len(df_pos)-sum(predictedPositive))
TN = sum(predictedNegative)
FN = (len(df_neg)-sum(predictedNegative))
print("Actual Positive: ",len(df_pos),"\tPredictedPositive(TP): ",TP,"\tFP: ",FP)
print("Actual Negative: ",len(df_neg),"\tPredictedNegative(TN): ",TN,"\tFN: ",FN)
F_measure = (2 * p * r)/(p+ r)
print("F-measure: "+str(F_measure))

Accuracy:  0.674 	Precision:  0.658631261738 	Recall:  0.746 	ROC:  0.674
Actual Positive:  500 	PredictedPositive(TP):  301 	FP:  199
Actual Negative:  500 	PredictedNegative(TN):  373 	FN:  127
F-measure: 0.699598442154


In [88]:
accuracy = []
precision= []
recall = []
roc = [] 
#df_pos = df_pos[0:700]
#df_neg = df_neg[0:700]
predictedPositive = []
predictedNegative = []
for train_index, test_index in kf.split(df_pos):
    X_train,y_train,X_test,y_test = get_train_test(df_pos,df_neg,train_index,test_index)
    X_train_vectorized,X_test_vectorized = feature_extraction(X_train,X_test)
    #model = LogisticRegression(C=100)
    #model = LogisticRegression(C=100, penalty='l1')
    model = LogisticRegression(C=100, penalty='l2')
    model.fit(X_train_vectorized,y_train)
    predictions = model.predict(X_test_vectorized)
    roc.append(roc_auc_score(y_test, predictions))
    accuracy.append(accuracy_score(y_test, predictions, normalize=True))
    precision.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions))
    ################################################
    #cm = confusion_matrix(y_test, predictions)
    #print(cm)
    #################################################
    result = pd.DataFrame.from_items([("Actual",y_test),("Prediction",predictions)])
    pp = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==0))]
    nn = [result['Actual'][x] for x in range(0,len(result)) if ((result['Actual'][x] ==result['Prediction'][x]) and (result['Actual'][x]==1))]
    predictedPositive.append(len(pp))
    predictedNegative.append(len(nn))   
    ##############################################################################################
    #print("Actual Positive: ",str(len(predictions)/2),"\tPredictedPositive(TP): ",str(len(pp)))
    #print("Actual Negative: ",str(len(predictions)/2),"\tPredictedNegative(TN): ",str(len(nn)))
    ###############################################################################################
a = sum(accuracy)/10
p = sum(precision)/10
r = sum(recall)/10
ro = sum(roc)/10
print("Accuracy: ",a,"\tPrecision: ",p,"\tRecall: ",r,"\tROC: ",ro)
TP = sum(predictedPositive)
FP = (len(df_pos)-sum(predictedPositive))
TN = sum(predictedNegative)
FN = (len(df_neg)-sum(predictedNegative))
print("Actual Positive: ",len(df_pos),"\tPredictedPositive(TP): ",TP,"\tFP: ",FP)
print("Actual Negative: ",len(df_neg),"\tPredictedNegative(TN): ",TN,"\tFN: ",FN)
F_measure = (2 * p * r)/(p+ r)
print("F-measure: "+str(F_measure))

Accuracy:  0.628 	Precision:  0.626711107124 	Recall:  0.642 	ROC:  0.628
Actual Positive:  500 	PredictedPositive(TP):  307 	FP:  193
Actual Negative:  500 	PredictedNegative(TN):  321 	FN:  179
F-measure: 0.634263432415


In [48]:
pd.set_option('display.width',2000)

In [143]:
df_pos['text'][0]

'@kpsharmaoli ओलीले कार्यकर्तालाई भने- पीडितको घरमा पाहुना नबन्नुस् #NepalEarthquake https://shar.es/1rI8n7\xa0pic.twitter.com/Uuus55ifhv '

In [144]:
df_neg['text'][0]

'कतारले नेपाली कामदारलाइ किरिया गर्न पनि घर जान दिएन http://goo.gl/CGkrLY\xa0 #NepalEarthquake #NepalRelief #NepalQuakeRelief '